In [1]:
import gct

In [2]:
prefix = "K_DoesAlgSupportDirTest"

def setup_data():
    gct.remove_data(prefix + '_*', dry_run=False)
    data1 = gct.random_dataset.generate_directed_unweighted_random_graph_LFR(name=prefix + "_1", \
                                   N=128, k=16, maxk=32, mu=0.2, minc=32)
    data2 = data1.as_undirected(newname=prefix + "_2")
    return data1,data2


In [3]:
data_w,data_uw=setup_data()

2020-04-27 23:15:16,961 - RandomGenerator:LFR - INFO - Runing '/opt/gct/submodules/CommunityDetectionCodes/lfr_dir_net -N 128 -k 16 -maxk 32 -mu 0.2 -minc 32'  with seed 498725
INFO:RandomGenerator:LFR:Runing '/opt/gct/submodules/CommunityDetectionCodes/lfr_dir_net -N 128 -k 16 -maxk 32 -mu 0.2 -minc 32'  with seed 498725
2020-04-27 23:15:16,962 - RandomGenerator:LFR - INFO - working dir: /tmp/tmp79uasbtf
INFO:RandomGenerator:LFR:working dir: /tmp/tmp79uasbtf
2020-04-27 23:15:16,998 - RandomGenerator:LFR - INFO - min node is 1. will make it 0
INFO:RandomGenerator:LFR:min node is 1. will make it 0
2020-04-27 23:15:17,007 - Dataset:K_DoesAlgSupportDirTest_1 - INFO - writing/data/data/K_DoesAlgSupportDirTest_1/edges.parq
INFO:Dataset:K_DoesAlgSupportDirTest_1:writing/data/data/K_DoesAlgSupportDirTest_1/edges.parq


Removing /data/data/K_DoesAlgSupportDirTest_1
Removing /data/result/K_DoesAlgSupportDirTest_1
Removing /data/data/K_DoesAlgSupportDirTest_2
Removing /data/result/K_DoesAlgSupportDirTest_2


2020-04-27 23:15:17,381 - Clustering - INFO - Writing /data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:Writing /data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
/opt/conda/envs/python3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
2020-04-27 23:15:17,392 - Dataset:K_DoesAlgSupportDirTest_2 - INFO - writing/data/data/K_DoesAlgSupportDirTest_2/edges.parq
INFO:Dataset:K_DoesAlgSupportDirTest_2:writing/data/data/K_DoesAlgSupportDirTest_2/edges.parq


In [11]:
data_w.name,data_uw.name

('K_DoesAlgSupportDirTest_1', 'K_DoesAlgSupportDirTest_2')

In [5]:
!pip install func-timeout

    100% |████████████████████████████████| 51kB 2.0MB/s ta 0:00:011
  Running setup.py bdist_wheel for func-timeout ... done
  Stored in directory: /root/.cache/pip/wheels/46/7c/4f/24f1d2d5bbff92219debe7ea19af84f76ddeb90dd4ec544f26
Successfully built func-timeout
You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import os
from func_timeout import func_timeout, FunctionTimedOut
from gct import config,utils
def has_run_1(runame, dsname):
    fpath = os.path.join(config.get_result_file_path(dsname=dsname, runname=runame), 'result.txt')
    return utils.file_exists(fpath)

def has_run(runame, dsname):
    fpath = os.path.join("/{}/tmp/{}_{}.done".format('data', runame,dsname))
    return utils.file_exists(fpath)

def timeout(func, args=(), kwargs={}, timeout_duration=10, default=None):
    result=default
    try:
        result = func_timeout(timeout_duration, func, args=args, kwargs=kwargs)
    except FunctionTimedOut:
        print("timeout", func,args,kwargs)
    except:
        print ("Unexpected error:", sys.exc_info()[0])
    return result

def timeout0(func, args=(), kwargs={}, timeout_duration=10, default=None):
    import signal

    class TimeoutError(Exception):
        pass

    def handler(signum, frame):
        raise TimeoutError()

    # set the timeout handler
    signal.signal(signal.SIGALRM, handler) 
    signal.alarm(timeout_duration)
    try:
        result = func(*args, **kwargs)
    except TimeoutError as exc:
        result = default
    finally:
        signal.alarm(0)

    return result


In [7]:

def run_alg(alg,dsname, check_skip=True):
    try:
        runname = alg
        if not check_skip or (check_skip and not has_run(runname, dsname)):
            print ("runing ", alg, dsname)
            if alg =='cdc_SVINET':
                gct.run_alg(runname=runname, data=gct.load_local_graph(dsname), algname=alg, seed=123, max_iterations=1000)        
            else:
                gct.run_alg(runname=runname, data=gct.load_local_graph(dsname), algname=alg, seed=123)
            print ("finished", alg,dsname)
        else:
            print("skip", runname,dsname)
    except:
        print("except", runname,dsname)
    return True
    

def run_alg_timeout(alg,dsname,timeout_duration=600):
    if not timeout(run_alg, (alg,dsname), timeout_duration=timeout_duration):
        print("tiemout", alg,dsname)
    fpath = os.path.join("/{}/tmp/{}_{}.done".format('data', alg,dsname))
    !touch {fpath}

        


In [15]:
run_alg("oslom_lpm", data_w.name)

2020-04-27 23:16:34,189 - Dataset:K_DoesAlgSupportDirTest_1 - INFO - writing edges to /data/data/K_DoesAlgSupportDirTest_1/edges.txt
INFO:Dataset:K_DoesAlgSupportDirTest_1:writing edges to /data/data/K_DoesAlgSupportDirTest_1/edges.txt
2020-04-27 23:16:34,191 - Dataset:K_DoesAlgSupportDirTest_1 - INFO - reading /data/data/K_DoesAlgSupportDirTest_1/edges.parq
INFO:Dataset:K_DoesAlgSupportDirTest_1:reading /data/data/K_DoesAlgSupportDirTest_1/edges.parq


runing  oslom_lpm K_DoesAlgSupportDirTest_1


2020-04-27 23:16:34,463 - Dataset:K_DoesAlgSupportDirTest_1 - INFO - finish writing /data/data/K_DoesAlgSupportDirTest_1/edges.txt
INFO:Dataset:K_DoesAlgSupportDirTest_1:finish writing /data/data/K_DoesAlgSupportDirTest_1/edges.txt
2020-04-27 23:16:34,464 - oslom_lpm - INFO - Running /opt/gct/submodules/OSLOM/lpm -f /tmp/tmp2q0fp_i7/edges.txt -r 1 -seed 123
INFO:oslom_lpm:Running /opt/gct/submodules/OSLOM/lpm -f /tmp/tmp2q0fp_i7/edges.txt -r 1 -seed 123
2020-04-27 23:16:34,478 - oslom_lpm - INFO - Made 5 clusters in 0.012883 seconds
INFO:oslom_lpm:Made 5 clusters in 0.012883 seconds


finished oslom_lpm K_DoesAlgSupportDirTest_1


True

In [16]:
run_alg_timeout("oslom_lpm", data_w.name)

2020-04-27 23:16:35,245 - oslom_lpm - INFO - Running /opt/gct/submodules/OSLOM/lpm -f /tmp/tmpz_kum25r/edges.txt -r 1 -seed 123
INFO:oslom_lpm:Running /opt/gct/submodules/OSLOM/lpm -f /tmp/tmpz_kum25r/edges.txt -r 1 -seed 123
2020-04-27 23:16:35,258 - oslom_lpm - INFO - Made 5 clusters in 0.011840 seconds
INFO:oslom_lpm:Made 5 clusters in 0.011840 seconds


runing  oslom_lpm K_DoesAlgSupportDirTest_1
finished oslom_lpm K_DoesAlgSupportDirTest_1


In [17]:
datasets=[data_w.name,data_uw.name]
for alg in gct.list_algorithms():
    for name in datasets:
        run_alg_timeout(alg, name)

2020-04-27 23:16:38,052 - oslom_Infohiermap - INFO - Running /opt/gct/submodules/OSLOM/infohiermap_dir 123 /tmp/tmpjn_0bmy7/pajek.txt 1
INFO:oslom_Infohiermap:Running /opt/gct/submodules/OSLOM/infohiermap_dir 123 /tmp/tmpjn_0bmy7/pajek.txt 1
2020-04-27 23:16:38,074 - oslom_Infohiermap - INFO - Made 6 clusters in 0.012223 seconds
INFO:oslom_Infohiermap:Made 6 clusters in 0.012223 seconds


runing  oslom_Infohiermap K_DoesAlgSupportDirTest_1
finished oslom_Infohiermap K_DoesAlgSupportDirTest_1


2020-04-27 23:16:38,188 - Dataset:K_DoesAlgSupportDirTest_2 - INFO - writing edges to /data/data/K_DoesAlgSupportDirTest_2/edges.txt
INFO:Dataset:K_DoesAlgSupportDirTest_2:writing edges to /data/data/K_DoesAlgSupportDirTest_2/edges.txt
2020-04-27 23:16:38,189 - Dataset:K_DoesAlgSupportDirTest_2 - INFO - reading /data/data/K_DoesAlgSupportDirTest_2/edges.parq
INFO:Dataset:K_DoesAlgSupportDirTest_2:reading /data/data/K_DoesAlgSupportDirTest_2/edges.parq


runing  oslom_Infohiermap K_DoesAlgSupportDirTest_2


2020-04-27 23:16:38,324 - Dataset:K_DoesAlgSupportDirTest_2 - INFO - finish writing /data/data/K_DoesAlgSupportDirTest_2/edges.txt
INFO:Dataset:K_DoesAlgSupportDirTest_2:finish writing /data/data/K_DoesAlgSupportDirTest_2/edges.txt
2020-04-27 23:16:38,342 - oslom_Infohiermap - INFO - Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmp1410n7gn/pajek.txt 1
INFO:oslom_Infohiermap:Running /opt/gct/submodules/OSLOM/infohiermap_undir 123 /tmp/tmp1410n7gn/pajek.txt 1
2020-04-27 23:16:38,368 - oslom_Infohiermap - INFO - Made 6 clusters in 0.017283 seconds
INFO:oslom_Infohiermap:Made 6 clusters in 0.017283 seconds


...

runing  alg_lso_cluster K_DoesAlgSupportDirTest_2
finished alg_lso_cluster K_DoesAlgSupportDirTest_2


2020-04-28 00:01:29,634 - alg_streamcom - INFO - Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportDirTest_1/edges.txt -o output /data/data/K_DoesAlgSupportDirTest_1/edges.txt
INFO:alg_streamcom:Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportDirTest_1/edges.txt -o output /data/data/K_DoesAlgSupportDirTest_1/edges.txt
2020-04-28 00:01:29,649 - alg_streamcom - INFO - Made 69 clusters in 0.012589 seconds
INFO:alg_streamcom:Made 69 clusters in 0.012589 seconds


runing  alg_streamcom K_DoesAlgSupportDirTest_1
finished alg_streamcom K_DoesAlgSupportDirTest_1


2020-04-28 00:01:29,770 - alg_streamcom - INFO - Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportDirTest_2/edges.txt -o output /data/data/K_DoesAlgSupportDirTest_2/edges.txt
INFO:alg_streamcom:Running /opt/gct/submodules/graph-streaming/streamcom --seed 123 -f /data/data/K_DoesAlgSupportDirTest_2/edges.txt -o output /data/data/K_DoesAlgSupportDirTest_2/edges.txt
2020-04-28 00:01:29,781 - alg_streamcom - INFO - Made 65 clusters in 0.009630 seconds
INFO:alg_streamcom:Made 65 clusters in 0.009630 seconds


runing  alg_streamcom K_DoesAlgSupportDirTest_2
finished alg_streamcom K_DoesAlgSupportDirTest_2


In [18]:
results = {}
d = gct.list_all_clustering_results(print_format=False)
for u in d:
    if prefix + "_" in u:
        for run in d[u]:
            a = gct.load_clustering_result(u, run)
            print (u, run)
            results[u + run] = a



K_DoesAlgSupportDirTest_1 oslom_lpm
K_DoesAlgSupportDirTest_1 oslom_Infohiermap
K_DoesAlgSupportDirTest_1 oslom_Infomap
K_DoesAlgSupportDirTest_1 oslom_OSLOM
K_DoesAlgSupportDirTest_1 oslom_copra
K_DoesAlgSupportDirTest_1 oslom_louvain_method
K_DoesAlgSupportDirTest_1 oslom_modopt
K_DoesAlgSupportDirTest_1 pycabem_GANXiSw
K_DoesAlgSupportDirTest_1 pycabem_HiReCS
K_DoesAlgSupportDirTest_1 pycabem_LabelRank
K_DoesAlgSupportDirTest_1 cdc_CONGA
K_DoesAlgSupportDirTest_1 cdc_Connected_Iterative_Scan
K_DoesAlgSupportDirTest_1 cdc_EAGLE
K_DoesAlgSupportDirTest_1 cdc_FastCpm
K_DoesAlgSupportDirTest_1 cdc_GCE
K_DoesAlgSupportDirTest_1 cdc_LinkCommunities
K_DoesAlgSupportDirTest_1 cdc_MOSES
K_DoesAlgSupportDirTest_1 cdc_MSCD_AFG
K_DoesAlgSupportDirTest_1 cdc_MSCD_HSLSW
K_DoesAlgSupportDirTest_1 cdc_MSCD_LFK
K_DoesAlgSupportDirTest_1 cdc_MSCD_LFK2
K_DoesAlgSupportDirTest_1 cdc_MSCD_RB
K_DoesAlgSupportDirTest_1 cdc_MSCD_SO
K_DoesAlgSupportDirTest_1 cdc_MSCD_SOM
K_DoesAlgSupportDirTest_1 cdc_ParCPM

In [19]:
import pandas as pd
lst = [];
for alg in  gct.list_algorithms():
    for dsname in datasets:
        if dsname.startswith(prefix + '_1'):
            a = [];columns = []
            dsname2 = dsname.replace(prefix + "_1", prefix + "_2")
            a += [alg, dsname, dsname2]
            columns += ['alg', 'data1', 'data2']
            if dsname + alg in results and dsname2 + alg in results:
                cluster1 = gct.to_cluster(results[dsname + alg])
                cluster2 = gct.to_cluster(results[dsname2 + alg])
                compa = gct.ClusterComparator(cluster1, cluster2)
                a += [compa.sklean_nmi(), cluster1.is_overlap, cluster2.is_overlap,
                      cluster1.num_cluster, cluster2.num_cluster,
                     compa.OvpNMI()['NMImax']]
                columns += ['nmi_12', 'ovp1', 'ovp2', "#c1", "#c2", 'ovpnmi_12']

                gt = list(gct.load_local_graph(dsname).get_ground_truth().values())[0]            
                compa = gct.ClusterComparator(gt, cluster1)            
                a += [compa.sklean_nmi(), compa.OvpNMI()['NMImax']]
                columns += ['nmi_t1', 'ovpnmi_t1']

                compa = gct.ClusterComparator(gt, cluster2)            
                a += [compa.sklean_nmi(), compa.OvpNMI()['NMImax']]
                columns += ['nmi_t2', 'ovpnmi_t2']

                lst.append(a)



2020-04-28 00:01:29,996 - ClusterComparator - INFO - resulting 128 nodes out of 129,128
INFO:ClusterComparator:resulting 128 nodes out of 129,128
2020-04-28 00:01:30,007 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp7zj3ekik/cluster1.cnl /tmp/tmp7zj3ekik/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp7zj3ekik/cluster1.cnl /tmp/tmp7zj3ekik/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:30,021 - ClusterComparator - ERROR - Run command with error status code 134
ERROR:ClusterComparator:Run command with error status code 134
2020-04-28 00:01:30,023 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
2020-04-28 00:01:30,029 - ClusterComparator - INFO - resulting 128 nodes out of 128,129
INFO:ClusterComparator:resulting 128 nodes out of 128,129
2020-04-28 00:01:30,035 - Clustering - INFO - persistent cluster

2020-04-28 00:01:30,834 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:30,844 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmptgwv__ih/cluster1.cnl /tmp/tmptgwv__ih/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmptgwv__ih/cluster1.cnl /tmp/tmptgwv__ih/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:30,856 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
2020-04-28 00:01:30,863 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:30,872 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpzlflu9n9/cluster1.cnl /tmp/tmpzlflu9n9/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/O

INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp81xbxh9t/cluster1.cnl /tmp/tmp81xbxh9t/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:31,583 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
2020-04-28 00:01:31,662 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:31,671 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp4d_1ghp3/cluster1.cnl /tmp/tmp4d_1ghp3/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp4d_1ghp3/cluster1.cnl /tmp/tmp4d_1ghp3/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:31,757 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:31,765 - ClusterComparator - INFO - Running /opt/gct/submodules/O

2020-04-28 00:01:33,368 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:33,381 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:33,389 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpbksgpp8_/cluster1.cnl /tmp/tmpbksgpp8_/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpbksgpp8_/cluster1.cnl /tmp/tmpbksgpp8_/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:33,401 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
2020-04-28 00:01:33,406 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:33,414 - ClusterComparator - INFO - Running /opt/gct/subm

2020-04-28 00:01:33,960 - ClusterComparator - ERROR - Run command with error status code 134
ERROR:ClusterComparator:Run command with error status code 134
2020-04-28 00:01:34,039 - ClusterComparator - INFO - resulting 127 nodes out of 128,128
INFO:ClusterComparator:resulting 127 nodes out of 128,128
2020-04-28 00:01:34,046 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpfpx5ot0w/cluster1.cnl /tmp/tmpfpx5ot0w/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpfpx5ot0w/cluster1.cnl /tmp/tmpfpx5ot0w/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:34,059 - ClusterComparator - ERROR - Run command with error status code 134
ERROR:ClusterComparator:Run command with error status code 134
2020-04-28 00:01:34,067 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:34,075 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/on

...

2020-04-28 00:01:35,307 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
INFO:Clustering:reading/data/data/K_DoesAlgSupportDirTest_1/gt_default.parq
2020-04-28 00:01:35,313 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:35,321 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpfrypyq71/cluster1.cnl /tmp/tmpfrypyq71/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpfrypyq71/cluster1.cnl /tmp/tmpfrypyq71/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:35,332 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:35,341 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmplyuf3i2v/cluster1.cnl /tmp/tmplyuf3i2v/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/O

2020-04-28 00:01:35,690 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:35,699 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpyph9_nfw/cluster1.cnl /tmp/tmpyph9_nfw/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpyph9_nfw/cluster1.cnl /tmp/tmpyph9_nfw/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:35,716 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:35,726 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpd809nnh1/cluster1.cnl /tmp/tmpd809nnh1/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpd809nnh1/cluster1.cnl /tmp/tmpd809nnh1/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:35,737 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_def

2020-04-28 00:01:36,071 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:36,079 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpd67_aypo/cluster1.cnl /tmp/tmpd67_aypo/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpd67_aypo/cluster1.cnl /tmp/tmpd67_aypo/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:36,097 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:36,106 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp0fdlu5b4/cluster1.cnl /tmp/tmp0fdlu5b4/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmp0fdlu5b4/cluster1.cnl /tmp/tmp0fdlu5b4/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:36,118 - Clustering - INFO - reading/data/data/K_DoesAlgSupportDirTest_1/gt_def

INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:36,454 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpof32288b/cluster1.cnl /tmp/tmpof32288b/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpof32288b/cluster1.cnl /tmp/tmpof32288b/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:36,465 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:36,473 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpq65y33_j/cluster1.cnl /tmp/tmpq65y33_j/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpq65y33_j/cluster1.cnl /tmp/tmpq65y33_j/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:36,490 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:36,499 - ClusterCompa

2020-04-28 00:01:36,821 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:36,829 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpagg37zjm/cluster1.cnl /tmp/tmpagg37zjm/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpagg37zjm/cluster1.cnl /tmp/tmpagg37zjm/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:36,839 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:ClusterComparator:resulting 128 nodes out of 128,128
2020-04-28 00:01:36,848 - ClusterComparator - INFO - Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpyq_1fjl6/cluster1.cnl /tmp/tmpyq_1fjl6/cluster2.cnl > ovpnmioutput
INFO:ClusterComparator:Running /opt/gct/submodules/OvpNMI/onmi /tmp/tmpyq_1fjl6/cluster1.cnl /tmp/tmpyq_1fjl6/cluster2.cnl > ovpnmioutput
2020-04-28 00:01:36,865 - ClusterComparator - INFO - resulting 128 nodes out of 128,128
INFO:Clu

In [20]:
rdf = pd.DataFrame(lst, columns=columns)
for alg in  gct.list_algorithms():
    if alg not in rdf['alg'].values:
        lst.append([alg] + [None] * (len(columns) - 1))
rdf = pd.DataFrame(lst, columns=columns)    
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    display (rdf)


,alg,data1,data2,nmi_12,ovp1,ovp2,#c1,#c2,ovpnmi_12,nmi_t1,ovpnmi_t1,nmi_t2,ovpnmi_t2
0,oslom_Infohiermap,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,8.019533e-02,False,False,6.0,6.0,NaN,8.019533e-02,NaN,1.000000e+00,1.000000
1,oslom_Infomap,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000e+00,False,False,6.0,6.0,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000
2,oslom_OSLOM,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000e+00,False,False,6.0,6.0,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000
3,oslom_copra,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,9.131541e-01,True,True,6.0,6.0,0.863149,9.817896e-01,0.983499,8.804805e-01,0.875279
4,oslom_louvain_method,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000e+00,False,False,6.0,6.0,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000
5,oslom_lpm,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000e+00,False,False,5.0,5.0,1.000000,9.385531e-01,0.847325,9.385531e-01,0.847325
6,oslom_modopt,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000e+00,False,False,6.0,6.0,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000
7,pycabem_GANXiSw,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,9.546494e-01,False,False,6.0,5.0,0.885584,1.000000e+00,1.000000,9.546494e-01,0.885584
8,pycabem_HiReCS,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,9.454628e-01,False,True,6.0,7.0,0.953739,8.362817e-02,NaN,9.354460e-02,NaN
9,pycabem_LabelRank,K_DoesAlgSupportDirTest_1,K_DoesAlgSupportDirTest_2,1.000000e+00,False,False,6.0,6.0,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000


In [11]:
run_alg("cdc_HDEMON", "K_DoesAlgSupportDirTest_2", check_skip=False)

2020-04-28 01:14:31,588 - cdc_HDEMON - INFO - seed ignored
INFO:cdc_HDEMON:seed ignored
2020-04-28 01:14:31,590 - cdc_HDEMON - INFO - Running python /opt/gct/submodules/CommunityDetectionCodes/Algorithms/2012-DEMON/HDemon.py edges.txt uw ud 0.25 5
INFO:cdc_HDEMON:Running python /opt/gct/submodules/CommunityDetectionCodes/Algorithms/2012-DEMON/HDemon.py edges.txt uw ud 0.25 5


runing  cdc_HDEMON K_DoesAlgSupportDirTest_2


2020-04-28 01:14:44,847 - cdc_HDEMON - INFO - Made 138 clusters with k=0
INFO:cdc_HDEMON:Made 138 clusters with k=0
2020-04-28 01:14:44,848 - cdc_HDEMON - INFO - Made 1 clusters with k=1
INFO:cdc_HDEMON:Made 1 clusters with k=1
2020-04-28 01:14:44,850 - cdc_HDEMON - INFO - Made 2 clusters in 13.254731 seconds
INFO:cdc_HDEMON:Made 2 clusters in 13.254731 seconds


finished cdc_HDEMON K_DoesAlgSupportDirTest_2


True